In [36]:
import numpy as np
import pandas as pd
import random
import math

In [61]:
#Zadanie 1 - klasa
class DataProcessing:
    @staticmethod
    def ShuffleData(X):
        for i in range(len(X)-1,0,-1):
            j = random.randint(0,i)
            X.iloc[i], X.iloc[j] = X.iloc[j], X.iloc[i]
        return X
    @staticmethod
    def NormalizeData(n):
        k = n.drop("Type", axis=1)
        k1 = (k - k.min())/(k.max()-k.min())
        for (name,col) in k1.iteritems():
            n[name] = col
        return n
    @staticmethod
    def SplitData(X,x):
        #y jest kalkulowane jako reszta (tzn. 100%-x)
        if x >= 10 or x < 0:
            raise ValueError('Niepoprawne x (musi być < 1)')
        return X[:math.ceil(len(X)*x)],X[math.ceil(len(X)*(1-x)):]

In [38]:
#Zadanie 2 - klasa
class NaiveBayes:
    #średnia dla atrybutu
    @staticmethod
    def mean(attr):
        try:
            return sum(attr)/len(attr)
        except TypeError:
            print(attr)
            return sum(attr)/len(attr)
    #odchylenie stand.
    def std(attr):
        mean = NaiveBayes.mean(attr)
        sumelem = 0
        for i in attr:
            sumelem += (i-mean)**2
        return math.sqrt(sumelem/len(attr))
    #obliczenie prawdopodobieństwa (funkcja gaussa)
    @staticmethod
    def gauss(x, mean, std):
        exp = np.exp(-(x-mean)**2/(2*std**2))
        return exp/(np.sqrt(2*np.pi*std**2))
    #klasyfikacja
    def classify(train,sample):
        #separacja klas z bazy X
        names = train.Type.unique()
        classes = []
        for name in names:
            classes += [train[train['Type'] == name]]
            del classes[-1]['Type']
        #obliczenie średniej/odchylenia dla każdego atrybutu i klasy
        #obliczenie składowych prawdopodobieństwa (gauss)
        classes_gauss = []
        for classy in classes:
            attrs_mean = []
            attrs_std = []
            attrs_gauss = []
            for (name, data) in classy.iteritems():
                attrs_mean += [NaiveBayes.mean(data.values)]
                attrs_std += [NaiveBayes.std(data.values)]
                attrs_gauss += [NaiveBayes.gauss(sample[name],attrs_mean[-1],attrs_std[-1])]
            classes_gauss += [math.prod(attrs_gauss)]
        #zwróć max
        return names[classes_gauss.index(max(classes_gauss))]

In [39]:
#Zadanie 3 - klasa
class SoftSet:
    @staticmethod
    def classify(sample, X, Y):
        results = []
        #calc
        for i in range(0,len(X)):
            results += [0]
            for key in sample:
                results[i] += sample[key]*Y[i][key]
        #print(results)
        #pick max entries
        ret = []
        max_res = max(results)
        for i in range(0,len(X)):
            if results[i] == max_res:
                ret += [X[i]]
        return ret

In [57]:
#Zadanie 4 - klasa
class Fuzzy:
    def __init__(self):
        self.antecedent=[]
        self.consequent=[]
        self.rules=[]
    def addAntecedent(self,parameter, linguisticValue,a,b):
        self.antecedent.append([parameter,linguisticValue,a,b])
    def addRule(self,rule):
        self.rules.append(rule)
    def addConsequent(self,linguisticValue,parameters):
        self.consequent.append([linguisticValue,parameters])
    def gaussFunction(self,x,m,b):
        return np.exp(-((x-m)**2)/(2*b**2))
    def compute(self,sample):
        #rozmycie
        fuzzyValues=[]
        for i in self.antecedent:
            fuzzyValues.append([i[0],i[1],self.gaussFunction(sample[i[0]],i[2],i[3])])
        #wnioskowanie
        rules_result = []
        for i in self.rules:
            tmp = 1
            for j in fuzzyValues:
                if i[j[0]] == j[1]:
                    tmp *= j[2]
                    rules_result.append(tmp)
        #wyostrzenie
        decisions=[]
        for j in rules_result:
            for i in self.consequent[0][1]:
                decisions.append([self.gaussFunction(j,i[1],i[2]),i[0]])
        maxx=0
        index=0
        j=0
        decision=''
        for i in decisions:
            if i[0] > maxx:
                maxx = i[0]
                index=j
                decision=i[1]
            j+=1
        print("Decyzja to:",decision,"oraz wynosi",maxx)
        return decision

In [64]:
#Zadanie 1 - test
seeds = pd.read_csv('seeds.csv')
print("<seeds>")
print(seeds.head())
seedsmixed = DataProcessing.ShuffleData(seeds)
print("<seedsmixed>")
print(seedsmixed.head())
seedsnormalized = DataProcessing.NormalizeData(seeds)
print("<seedsnormalized>")
print(seedsnormalized.head())
a, b = DataProcessing.SplitData(seedsmixed,0.7)
print("total length:",len(seedsmixed))
print("split a length:",len(a))
print("split b length:",len(b))

<seeds>
    Area  Perimeter  Compactness  Kernel.Length  Kernel.Width  \
0  15.26      14.84       0.8710          5.763         3.312   
1  14.88      14.57       0.8811          5.554         3.333   
2  14.29      14.09       0.9050          5.291         3.337   
3  13.84      13.94       0.8955          5.324         3.379   
4  16.14      14.99       0.9034          5.658         3.562   

   Asymmetry.Coeff  Kernel.Groove  Type  
0            2.221          5.220     1  
1            1.018          4.956     1  
2            2.699          4.825     1  
3            2.259          4.805     1  
4            1.355          5.175     1  
<seedsmixed>
    Area  Perimeter  Compactness  Kernel.Length  Kernel.Width  \
0  15.26      14.84        0.871          5.763         3.312   
1  15.26      14.84        0.871          5.763         3.312   
2  15.26      14.84        0.871          5.763         3.312   
3  15.26      14.84        0.871          5.763         3.312   
4  15.26   

In [48]:
#Zadanie 2 - test
seeds = pd.read_csv('seeds.csv')
seedsmixed = DataProcessing.ShuffleData(seeds)
seedtrain, seedvalid = DataProcessing.SplitData(seedsmixed,0.7)
correct = 0
for i in range(0,len(seedvalid)):
    sample = seedvalid.iloc[i].drop('Type').to_dict()
    if seedvalid.iloc[i].Type == NaiveBayes.classify(seedtrain,sample):
        correct += 1
accuracy = correct/len(seedvalid.index)*100
print("Accuracy (regular) -",accuracy,"%")

Accuracy (regular) - 90.64748201438849 %


In [167]:
#Zadanie 2 - test
seeds = pd.read_csv('seeds.csv')
seeds = DataProcessing.ShuffleData(seeds)
seeds = DataProcessing.NormalizeData(seeds)
seedtrain, seedvalid = DataProcessing.SplitData(seeds,0.7)
correct = 0
for i in range(0,len(seedvalid)):
    sample = seedvalid.iloc[i].drop('Type').to_dict()
    if seedvalid.iloc[i].Type == NaiveBayes.classify(seedtrain,sample):
        correct += 1
accuracy = correct/len(seedvalid.index)*100
print("Accuracy (normalized) -",accuracy,"%")

Accuracy (normalized) - 92.80575539568345 %


In [240]:
#Zadanie 3 - test
seeds = pd.read_csv('seeds.csv')
seeds = DataProcessing.ShuffleData(seeds)
seedtrain, seedvalid = DataProcessing.SplitData(seeds,0.7)

X = []
Y = []
global_avgs = {}
for (column_name, column_data) in seedtrain.iteritems():
    if column_name == "Type":
        continue
    global_avgs[column_name] = sum(column_data)/len(column_data)
for name in seedtrain["Type"].unique():
    X += [name]
    frame = seedtrain[seedtrain["Type"] == name]
    frame.pop("Type")
    y = {}
    for (column_name, column_data) in frame.iteritems():
        avg = sum(column_data)/len(column_data)
        y[column_name] = 1 if avg >= global_avgs[column_name] else 0
    Y += [y]
    
correct = 0
for i in range(0,len(seedvalid)):
    sample = seedvalid.iloc[i].drop('Type').to_dict()
    if seedvalid.iloc[i].Type in SoftSet.classify(sample,X,Y):
        correct += 1
accuracy = correct/len(seedvalid.index)*100
print("Accuracy -",accuracy,"%")

Accuracy - 33.093525179856115 %


In [65]:
#Zadanie 4 - test
system = Fuzzy()

#poprzednicy i następnicy
#3 wykresy, obecne wartości zostały wybrane by w miarę podzielić przedział 0-1 na 3 części
system.addAntecedent('temperatura','poor',0.33,0.1)
system.addAntecedent('temperatura','avg',0.60,0.1)
system.addAntecedent('temperatura','good',0.8,0.1)
system.addAntecedent('wilgotnosc','poor',0.8,0.1)
system.addAntecedent('wilgotnosc','avg',0.55,0.1)
system.addAntecedent('wilgotnosc','good',0.33,0.1)
system.addAntecedent('cisnienie','poor',0.33,0.1)
system.addAntecedent('cisnienie','avg',0.8,0.1)
system.addAntecedent('cisnienie','good',0.6,0.1)
system.addAntecedent('zanieczyszczenie','poor',0.8,0.1)
system.addAntecedent('zanieczyszczenie','avg',0.6,0.1)
system.addAntecedent('zanieczyszczenie','good',0.33,0.1)

system.addConsequent('result',[['poor',0.33,0.2],['avg',0.6,0.2],['good',1,0.2]])

#reguły
system.addRule({'temperatura':'poor','wilgotnosc':'poor','cisnienie':'poor','zanieczyszczenie':'poor','result':'poor'})
system.addRule({'temperatura':'poor','wilgotnosc':'poor','cisnienie':'poor','zanieczyszczenie':'avg','result':'poor'})
system.addRule({'temperatura':'poor','wilgotnosc':'poor','cisnienie':'poor','zanieczyszczenie':'good','result':'poor'})
system.addRule({'temperatura':'poor','wilgotnosc':'poor','cisnienie':'avg','zanieczyszczenie':'poor','result':'poor'})
system.addRule({'temperatura':'poor','wilgotnosc':'poor','cisnienie':'avg','zanieczyszczenie':'avg','result':'poor'})
system.addRule({'temperatura':'poor','wilgotnosc':'poor','cisnienie':'avg','zanieczyszczenie':'good','result':'poor'})
system.addRule({'temperatura':'poor','wilgotnosc':'poor','cisnienie':'good','zanieczyszczenie':'poor','result':'poor'})
system.addRule({'temperatura':'poor','wilgotnosc':'poor','cisnienie':'good','zanieczyszczenie':'avg','result':'poor'})
system.addRule({'temperatura':'poor','wilgotnosc':'poor','cisnienie':'good','zanieczyszczenie':'good','result':'avg'})
system.addRule({'temperatura':'poor','wilgotnosc':'avg','cisnienie':'poor','zanieczyszczenie':'poor','result':'poor'})
system.addRule({'temperatura':'poor','wilgotnosc':'avg','cisnienie':'poor','zanieczyszczenie':'avg','result':'poor'})
system.addRule({'temperatura':'poor','wilgotnosc':'avg','cisnienie':'poor','zanieczyszczenie':'good','result':'avg'})
system.addRule({'temperatura':'poor','wilgotnosc':'avg','cisnienie':'avg','zanieczyszczenie':'poor','result':'poor'})
system.addRule({'temperatura':'poor','wilgotnosc':'avg','cisnienie':'avg','zanieczyszczenie':'avg','result':'avg'})
system.addRule({'temperatura':'poor','wilgotnosc':'avg','cisnienie':'avg','zanieczyszczenie':'good','result':'avg'})
system.addRule({'temperatura':'poor','wilgotnosc':'avg','cisnienie':'good','zanieczyszczenie':'poor','result':'poor'})
system.addRule({'temperatura':'poor','wilgotnosc':'avg','cisnienie':'good','zanieczyszczenie':'avg','result':'avg'})
system.addRule({'temperatura':'poor','wilgotnosc':'avg','cisnienie':'good','zanieczyszczenie':'good','result':'good'})
system.addRule({'temperatura':'poor','wilgotnosc':'good','cisnienie':'poor','zanieczyszczenie':'poor','result':'poor'})
system.addRule({'temperatura':'poor','wilgotnosc':'good','cisnienie':'poor','zanieczyszczenie':'avg','result':'poor'})
system.addRule({'temperatura':'poor','wilgotnosc':'good','cisnienie':'poor','zanieczyszczenie':'good','result':'poor'})
system.addRule({'temperatura':'poor','wilgotnosc':'good','cisnienie':'avg','zanieczyszczenie':'poor','result':'poor'})
system.addRule({'temperatura':'poor','wilgotnosc':'good','cisnienie':'avg','zanieczyszczenie':'avg','result':'avg'})
system.addRule({'temperatura':'poor','wilgotnosc':'good','cisnienie':'avg','zanieczyszczenie':'good','result':'good'})
system.addRule({'temperatura':'poor','wilgotnosc':'good','cisnienie':'good','zanieczyszczenie':'poor','result':'avg'})
system.addRule({'temperatura':'poor','wilgotnosc':'good','cisnienie':'good','zanieczyszczenie':'avg','result':'good'})
system.addRule({'temperatura':'poor','wilgotnosc':'good','cisnienie':'good','zanieczyszczenie':'good','result':'good'})
system.addRule({'temperatura':'avg','wilgotnosc':'poor','cisnienie':'poor','zanieczyszczenie':'poor','result':'poor'})
system.addRule({'temperatura':'avg','wilgotnosc':'poor','cisnienie':'poor','zanieczyszczenie':'avg','result':'poor'})
system.addRule({'temperatura':'avg','wilgotnosc':'poor','cisnienie':'poor','zanieczyszczenie':'good','result':'poor'})
system.addRule({'temperatura':'avg','wilgotnosc':'poor','cisnienie':'avg','zanieczyszczenie':'poor','result':'poor'})
system.addRule({'temperatura':'avg','wilgotnosc':'poor','cisnienie':'avg','zanieczyszczenie':'avg','result':'avg'})
system.addRule({'temperatura':'avg','wilgotnosc':'poor','cisnienie':'avg','zanieczyszczenie':'good','result':'avg'})
system.addRule({'temperatura':'avg','wilgotnosc':'poor','cisnienie':'good','zanieczyszczenie':'poor','result':'poor'})
system.addRule({'temperatura':'avg','wilgotnosc':'poor','cisnienie':'good','zanieczyszczenie':'avg','result':'avg'})
system.addRule({'temperatura':'avg','wilgotnosc':'poor','cisnienie':'good','zanieczyszczenie':'good','result':'good'})
system.addRule({'temperatura':'avg','wilgotnosc':'avg','cisnienie':'poor','zanieczyszczenie':'poor','result':'poor'})
system.addRule({'temperatura':'avg','wilgotnosc':'avg','cisnienie':'poor','zanieczyszczenie':'avg','result':'poor'})
system.addRule({'temperatura':'avg','wilgotnosc':'avg','cisnienie':'poor','zanieczyszczenie':'good','result':'avg'})
system.addRule({'temperatura':'avg','wilgotnosc':'avg','cisnienie':'avg','zanieczyszczenie':'poor','result':'avg'})
system.addRule({'temperatura':'avg','wilgotnosc':'avg','cisnienie':'avg','zanieczyszczenie':'avg','result':'avg'})
system.addRule({'temperatura':'avg','wilgotnosc':'avg','cisnienie':'avg','zanieczyszczenie':'good','result':'good'})
system.addRule({'temperatura':'avg','wilgotnosc':'avg','cisnienie':'good','zanieczyszczenie':'poor','result':'avg'})
system.addRule({'temperatura':'avg','wilgotnosc':'avg','cisnienie':'good','zanieczyszczenie':'avg','result':'avg'})
system.addRule({'temperatura':'avg','wilgotnosc':'avg','cisnienie':'good','zanieczyszczenie':'good','result':'good'})
system.addRule({'temperatura':'avg','wilgotnosc':'good','cisnienie':'poor','zanieczyszczenie':'poor','result':'poor'})
system.addRule({'temperatura':'avg','wilgotnosc':'good','cisnienie':'poor','zanieczyszczenie':'avg','result':'avg'})
system.addRule({'temperatura':'avg','wilgotnosc':'good','cisnienie':'poor','zanieczyszczenie':'good','result':'avg'})
system.addRule({'temperatura':'avg','wilgotnosc':'good','cisnienie':'avg','zanieczyszczenie':'poor','result':'avg'})
system.addRule({'temperatura':'avg','wilgotnosc':'good','cisnienie':'avg','zanieczyszczenie':'avg','result':'avg'})
system.addRule({'temperatura':'avg','wilgotnosc':'good','cisnienie':'avg','zanieczyszczenie':'good','result':'good'})
system.addRule({'temperatura':'avg','wilgotnosc':'good','cisnienie':'good','zanieczyszczenie':'poor','result':'avg'})
system.addRule({'temperatura':'avg','wilgotnosc':'good','cisnienie':'good','zanieczyszczenie':'avg','result':'good'})
system.addRule({'temperatura':'avg','wilgotnosc':'good','cisnienie':'good','zanieczyszczenie':'good','result':'good'})
system.addRule({'temperatura':'good','wilgotnosc':'poor','cisnienie':'poor','zanieczyszczenie':'poor','result':'poor'})
system.addRule({'temperatura':'good','wilgotnosc':'poor','cisnienie':'poor','zanieczyszczenie':'avg','result':'poor'})
system.addRule({'temperatura':'good','wilgotnosc':'poor','cisnienie':'poor','zanieczyszczenie':'good','result':'poor'})
system.addRule({'temperatura':'good','wilgotnosc':'poor','cisnienie':'avg','zanieczyszczenie':'poor','result':'poor'})
system.addRule({'temperatura':'good','wilgotnosc':'poor','cisnienie':'avg','zanieczyszczenie':'avg','result':'avg'})
system.addRule({'temperatura':'good','wilgotnosc':'poor','cisnienie':'avg','zanieczyszczenie':'good','result':'avg'})
system.addRule({'temperatura':'good','wilgotnosc':'poor','cisnienie':'good','zanieczyszczenie':'poor','result':'avg'})
system.addRule({'temperatura':'good','wilgotnosc':'poor','cisnienie':'good','zanieczyszczenie':'avg','result':'good'})
system.addRule({'temperatura':'good','wilgotnosc':'poor','cisnienie':'good','zanieczyszczenie':'good','result':'good'})
system.addRule({'temperatura':'good','wilgotnosc':'avg','cisnienie':'poor','zanieczyszczenie':'poor','result':'poor'})
system.addRule({'temperatura':'good','wilgotnosc':'avg','cisnienie':'poor','zanieczyszczenie':'avg','result':'avg'})
system.addRule({'temperatura':'good','wilgotnosc':'avg','cisnienie':'poor','zanieczyszczenie':'good','result':'avg'})
system.addRule({'temperatura':'good','wilgotnosc':'avg','cisnienie':'avg','zanieczyszczenie':'poor','result':'avg'})
system.addRule({'temperatura':'good','wilgotnosc':'avg','cisnienie':'avg','zanieczyszczenie':'avg','result':'good'})
system.addRule({'temperatura':'good','wilgotnosc':'avg','cisnienie':'avg','zanieczyszczenie':'good','result':'good'})
system.addRule({'temperatura':'good','wilgotnosc':'avg','cisnienie':'good','zanieczyszczenie':'poor','result':'avg'})
system.addRule({'temperatura':'good','wilgotnosc':'avg','cisnienie':'good','zanieczyszczenie':'avg','result':'good'})
system.addRule({'temperatura':'good','wilgotnosc':'avg','cisnienie':'good','zanieczyszczenie':'good','result':'good'})
system.addRule({'temperatura':'good','wilgotnosc':'good','cisnienie':'poor','zanieczyszczenie':'poor','result':'poor'})
system.addRule({'temperatura':'good','wilgotnosc':'good','cisnienie':'poor','zanieczyszczenie':'avg','result':'avg'})
system.addRule({'temperatura':'good','wilgotnosc':'good','cisnienie':'poor','zanieczyszczenie':'good','result':'good'})
system.addRule({'temperatura':'good','wilgotnosc':'good','cisnienie':'avg','zanieczyszczenie':'poor','result':'avg'})
system.addRule({'temperatura':'good','wilgotnosc':'good','cisnienie':'avg','zanieczyszczenie':'avg','result':'good'})
system.addRule({'temperatura':'good','wilgotnosc':'good','cisnienie':'avg','zanieczyszczenie':'good','result':'good'})
system.addRule({'temperatura':'good','wilgotnosc':'good','cisnienie':'good','zanieczyszczenie':'poor','result':'avg'})
system.addRule({'temperatura':'good','wilgotnosc':'good','cisnienie':'good','zanieczyszczenie':'avg','result':'good'})
system.addRule({'temperatura':'good','wilgotnosc':'good','cisnienie':'good','zanieczyszczenie':'good','result':'good'})

sample={'temperatura':0.1,'wilgotnosc':0.5,'cisnienie':0.1,'zanieczyszczenie':0.1}
system.compute(sample)
sample={'temperatura':0.8,'wilgotnosc':0.5,'cisnienie':0.3,'zanieczyszczenie':0.4}
system.compute(sample)
sample={'temperatura':0.5,'wilgotnosc':0.5,'cisnienie':0.4,'zanieczyszczenie':0.3}
system.compute(sample)
sample={'temperatura':0.1,'wilgotnosc':0.1,'cisnienie':0.6,'zanieczyszczenie':0.1}
system.compute(sample)
sample={'temperatura':0.5,'wilgotnosc':0.5,'cisnienie':0.2,'zanieczyszczenie':0.5}
system.compute(sample)
sample={'temperatura':0.8,'wilgotnosc':0.33,'cisnienie':0.6,'zanieczyszczenie':0.33}
system.compute(sample)

Decyzja to: poor oraz wynosi 0.43236815851128907
Decyzja to: good oraz wynosi 1.0
Decyzja to: avg oraz wynosi 0.9994670231291315
Decyzja to: poor oraz wynosi 0.43236815851128907
Decyzja to: avg oraz wynosi 0.9994670231291315
Decyzja to: good oraz wynosi 1.0


'good'